# 利用Attention机制来加强我们的翻译效果

上次的训练结果，大家也看到了 [使用Encoder-Decoder来完成机器翻译](https://github.com/LianHaiMiao/pytorch-lesson-zh/blob/master/NLP/encode_decoder.ipynb) ，很不幸，我们训练出了一个智障，那么有没有什么办法，可以提高它的“智商”呢？让它的翻译效果稍微提升一丢丢呢？

有的！ 这就是 **attention 机制**。

在上一篇中，我们的 decoder 在各个时刻使用了相同的背景向量。但是，如果解码器可以在不同时刻使用不同的背景向量呢，效果会不会更好呢？

以 英语-中文 为例子，给定一个输入序列 "I Love You" 和输出序列 "我爱你" ，解码器在 t1 时刻可以使用更多编码了 “I” 的信息去解码生成 "我" ，在 t2 时刻可以使用更多编码了 "Love" 的信息去解码生成 "爱"。这听起来就像是解码器在不同的时刻对输入的数据有着不同的 “注意力” 这也就是注意力机制 (attention) 的由来。

此时，相比于前一章节的模型，我们只需要更改 Decoder 部分的代码。

此时 Decoder 模型的示意图是：


![Decoder with attention](./images/attention-decoder-network.png)


步骤基本跟前面的 encoder-decoder 类似，仅仅需要少量的改动

## 第一步：构建一个Config类，用于保存各种超参数，以及导入各种包

In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
from torch import optim
import torch.nn.functional as F
import unicodedata, string, re, random, time, math

In [25]:
class Config():
    def __init__(self):
        self.data_path = "../data/cmn-eng/cmn.txt" # 数据放在 /data 目录下
        self.use_gpu = True
        self.hidden_size = 64
        self.encoder_lr = 5*1e-5
        self.decoder_lr = 5*1e-5
        self.train_num = 100000 # 训练数据集的数目
        self.print_epoch = 10000
        self.MAX_Len = 15
config = Config()

## 第二步：数据预处理

准备数据的全部过程如下所示：

1. 读取txt文件，并按行分割，再把每一行分割成一个pair (Eng, Chinese)
2. 过滤并处理文本信息
3. 从每个pair中，制作出 中文词典 和 英文词典
4. 构建训练集

data下载地址为： http://www.manythings.org/anki/cmn-eng.zip

该数据集中还有其他类型的翻译数据 http://www.manythings.org/anki/


——————————————————————

**这里需要注意，当我们下载完成之后，我们要把数据放在主目录下的 /data 文件夹下**

格式：/data/cmn-eng/cmn.txt


——————————————————————


中文词典和英文词典，我们使用*Lang* 类，该类包含了所有的 中文（英文） -> 数字 或者 数字 -> 中文（英文）的映射。

同时，我们要给一句话的其实和结束加上标志符

起始符：(Start Of Sentence)

SOS_token = 0

结束符：(End Of Sentence)

EOS_token = 1

另外，在这个类中，我们需要添加一个 *word2count* 方法，用来计算各个词出现的次数



In [3]:
SOS_token = 0
EOS_token = 1

class Lang():
    def __init__(self, name):
        self.name = name
        self.word2index = {}
        self.index2word = {0: "SOS", 1: "EOS"}
        self.word2count = {}
        self.n_words = 2  # Count SOS and EOS
    
    def addSentence(self, sentence):
        if self.name == "Chinese":
            for word in sentence:
                self.addWord(word)
        else:
            for word in sentence.split(' '):
                self.addWord(word)
    
    def addWord(self, word):
        if word not in self.word2index:
            self.word2index[word] = self.n_words
            self.word2count[word] = 1
            self.index2word[self.n_words] = word
            self.n_words += 1
        else:
            self.word2count[word] += 1

In [4]:
def unicodeToAscii(s):
    return ''.join(
        c for c in unicodedata.normalize('NFD', s)
        if unicodedata.category(c) != 'Mn'
    )

# Lowercase, trim, and remove non-letter characters
def normalizeString(s):
    s = unicodeToAscii(s.lower().strip())
    s = re.sub(r"([.!?])", r" \1", s)
    s = re.sub(r"[^a-zA-Z.!?]+", r" ", s)
    return s

In [5]:
def readLangs(lang1, lang2, pairs_file, reverse=False):
    print("Reading lines...")

    # Read the file and split into lines
    lines = open(pairs_file, encoding='utf-8').read().strip().split('\n')
    # Split every line into pairs and normalize
    pairs = []
    for l in lines:
        temp = l.split('\t')
        eng_unit = normalizeString(temp[0])
        chinese_unit = temp[1]
        pairs.append([eng_unit, chinese_unit])
    
    # Reverse pairs, make Lang instances
    if reverse:
        pairs = [list(reversed(p)) for p in pairs]
        input_lang = Lang(lang2)
        output_lang = Lang(lang1)
    else:
        input_lang = Lang(lang1)
        output_lang = Lang(lang2)
        
    return input_lang, output_lang, pairs

In [6]:
MAX_LENGTH = config.MAX_Len  # 长度大于15的我们统统舍弃

eng_prefixes = (
    "i am ", "i m ",
    "he is", "he s ",
    "she is", "she s",
    "you are", "you re ",
    "we are", "we re ",
    "they are", "they re ",
    "i", "he", 'you', 'she', 'we',
    'they', 'it'
)

def filterPair(p):
    return len(p[0].split(' ')) < MAX_LENGTH and \
        len(p[1]) < MAX_LENGTH and \
        p[0].startswith(eng_prefixes)

def filterPairs(pairs):
    return [pair for pair in pairs if filterPair(pair)]

In [7]:
def prepareData(lang1, lang2, pairs_file, reverse=False):
    input_lang, output_lang, pairs = readLangs(lang1, lang2, pairs_file, reverse)
    print("Read %s sentence pairs" % len(pairs))
    pairs = filterPairs(pairs)
    print("Trimmed to %s sentence pairs" % len(pairs))
    print("Counting words...")
    for pair in pairs:
        input_lang.addSentence(pair[0])
        output_lang.addSentence(pair[1])
    print("Counted words:")
    print(input_lang.name, "字典的大小为", str(input_lang.n_words))
    print(output_lang.name, "字典的大小为", str(output_lang.n_words))
    return input_lang, output_lang, pairs

input_lang, output_lang, pairs = prepareData('Eng', 'Chinese', config.data_path)
print(random.choice(pairs))


Reading lines...
Read 19777 sentence pairs
Trimmed to 9473 sentence pairs
Counting words...
Counted words:
Eng 字典的大小为 3737
Chinese 字典的大小为 2638
['you are not coming are you ?', '你不會來，是嗎？']


**到目前为止，我们已经把字典构建好了，接下来就是构建训练集**

In [8]:
def indexesFromSentence(lang, sentence):
    if lang.name == "Chinese":
        return [lang.word2index[word] for word in sentence]
    else:
        return [lang.word2index[word] for word in sentence.split(' ')]

def variableFromSentence(lang, sentence, use_gpu):
    indexes = indexesFromSentence(lang, sentence)
    indexes.append(EOS_token)
    result = Variable(torch.LongTensor(indexes).view(-1, 1)) # seq*1
    if use_gpu:
        return result.cuda()
    else:
        return result

def variablesFromPair(pair, use_gpu):
    input_variable = variableFromSentence(input_lang, pair[0], use_gpu)
    target_variable = variableFromSentence(output_lang, pair[1], use_gpu)
    return (input_variable, target_variable)

In [9]:
# 随机获取2个训练数据集， 这里我们依旧不用进行 batch 处理，下一章节 attention 机制中，我们再进行 batch 处理
example_pairs = [variablesFromPair(random.choice(pairs), config.use_gpu)
                      for i in range(2)]
print(example_pairs)

[(Variable containing:
    4
 1075
   26
  413
 1224
    6
    1
[torch.cuda.LongTensor of size 7x1 (GPU 0)]
, Variable containing:
    6
 1164
    2
  206
  850
  419
 1166
    4
    1
[torch.cuda.LongTensor of size 9x1 (GPU 0)]
), (Variable containing:
   38
 2390
  192
  554
  383
    6
    1
[torch.cuda.LongTensor of size 7x1 (GPU 0)]
, Variable containing:
   47
   42
  308
  265
  230
  309
    9
  189
    4
    1
[torch.cuda.LongTensor of size 10x1 (GPU 0)]
)]


## 第三步：构建编码器

编码器的结构，如图所示：


![encoder-network](./images/encoder-network.png)



In [10]:
class Encoder(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(Encoder, self).__init__()
        self.hidden_size = hidden_size
        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
    
    def forward(self, x, hidden):
        embedded = self.embedding(x).view(1, x.size()[0], -1)
        output = embedded  # batch*seq*feature
        output, hidden = self.gru(output, hidden)
        return output, hidden
    
    def initHidden(self, use_gpu):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result

## 第四步：构建解码器

编码器的结构，如图所示：

![Decoder with attention](./images/attention-decoder-network.png)


**todo_list: 图片中的模型，跟我们这里构建的模型有差异。以后记得补上我们这里的模型图。**


In [66]:
class AttentionDecoder(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=MAX_LENGTH):
        super(AttentionDecoder, self).__init__()
        self.hidden_size = hidden_size
        self.dropout_p = dropout_p
        self.max_length = max_length
        
        self.embedding = nn.Embedding(output_size, hidden_size)
        
        # attention 机制
        self.attn = nn.Sequential(
            nn.Linear(self.hidden_size * 2, self.max_length),
            nn.Tanh(),
            nn.Linear(self.max_length, 1)
        )
        
        # 结合之后的值
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        
        # drop out 防止过拟合
        self.dropout = nn.Dropout(self.dropout_p)
        
        self.gru = nn.GRU(hidden_size, hidden_size, batch_first=True)
        self.out = nn.Linear(hidden_size, output_size)
        self.softmax = nn.LogSoftmax()

    def forward(self, x, hidden, encoder_outputs):
        """
            x: 1*1
            hidden: 1*1*embed_size
            encoder_outputs: 1*seq_len*embed_size
        """
        cur_input_data = self.embedding(x).view(1, 1, -1) # 1*1*embed_size
        
        cur_seq_len = encoder_outputs.size()[1]
        hidden_broadcast = hidden.expand(1, cur_seq_len, self.hidden_size)
        
        # concate 操作根据 hidden 和 encoder_outputs 来求出当前context环境中的权重
        encoder_outputs_and_hiddens = torch.cat((encoder_outputs, hidden_broadcast), dim=2)

        # 计算 attention weights
        attn_weights = F.softmax(
            self.attn(encoder_outputs_and_hiddens)) # size: 1 * seq_len * 1
        
        decoder_context = torch.bmm(attn_weights.view(1, 1, -1), encoder_outputs) # size: 1*1*embed_size
        
        # 把 context 和 input 结合起来
        input_and_context = torch.cat((cur_input_data, decoder_context), dim=2) # size: 1*1*(embed_size+embed_size)
        
        concat_input = self.attn_combine(input_and_context) # size: 1*1*embed_size
      
        output, hidden = self.gru(concat_input, hidden)
        output = self.softmax(self.out(output[0]))
        return output, hidden, attn_weights

    def initHidden(self, use_gpu):
        result = Variable(torch.zeros(1, 1, self.hidden_size))
        if use_gpu:
            return result.cuda()
        else:
            return result

## 第五步：开始训练

定义优化器、损失函数，然后开始进行训练


In [67]:
# 实例化模型

encoder = Encoder(input_lang.n_words, config.hidden_size)
encoder = encoder.cuda() if config.use_gpu else encoder

attention_decoder = AttentionDecoder(config.hidden_size, input_lang.n_words)
attention_decoder = attention_decoder.cuda() if config.use_gpu else attention_decoder

# 定义优化器

encoder_optimizer = optim.Adam(encoder.parameters(), lr=config.encoder_lr)

decoder_optimizer = optim.Adam(attention_decoder.parameters(), lr=config.decoder_lr)


# 定义损失函数

fn_loss = nn.NLLLoss()

training_pairs = [variablesFromPair(random.choice(pairs), config.use_gpu)
                      for i in range(config.train_num)]

In [69]:
# 开始训练
for iter in range(1, config.train_num+1):
    training_pair = training_pairs[iter - 1]
    input_variable = training_pair[0]  # seq_len * 1
    target_variable = training_pair[1]  # seq_len * 1
    
    loss = 0
    
    # 训练过程
    encoder_hidden = encoder.initHidden(config.use_gpu)
    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_variable.size()[0]
    target_length = target_variable.size()[0]
    
    # 传入 encoder
    encoder_output, encoder_hidden = encoder(input_variable, encoder_hidden)
    
    # decoder 起始
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if config.use_gpu else decoder_input
    
    decoder_hidden = encoder_hidden
    
    for di in range(target_length):
        print(decoder_input)
        decoder_output, decoder_hidden, decoder_attention = attention_decoder(decoder_input, decoder_hidden, encoder_output)
        targ = target_variable[di]
        loss += fn_loss(decoder_output, targ)
        decoder_input = targ
    
    # 反向求导
    loss.backward()
    # 更新梯度
    encoder_optimizer.step()
    decoder_optimizer.step()
    
    print_loss = loss.data[0] / target_length
    
    if iter % config.print_epoch == 0:
        print("loss is: %.4f" % (print_loss))

Variable containing:
 0
[torch.cuda.LongTensor of size 1x1 (GPU 0)]

Variable containing:
 56
[torch.cuda.LongTensor of size 1 (GPU 0)]



RuntimeError: invalid argument 2: out of range at /opt/conda/conda-bld/pytorch_1503970438496/work/torch/lib/THC/generic/THCTensor.c:23

## 第六步：随机采样，对模型进行测试

In [64]:
def sampling(encoder, decoder):
    # 随机选择一个句子
    pair = random.choice(pairs)
    print('>', pair[0])
    print('=', pair[1])
    # 扔进模型中，进行翻译
    input_variable = variableFromSentence(input_lang, pair[0], config.use_gpu)
    input_length = input_variable.size()[0]
    encoder_hidden = encoder.initHidden(config.use_gpu)
    encoder_output, encoder_hidden = encoder(input_variable, encoder_hidden)
    
    decoder_input = Variable(torch.LongTensor([[SOS_token]]))
    decoder_input = decoder_input.cuda() if config.use_gpu else decoder_input
    decoder_hidden = encoder_hidden
    
    decoded_words = []
    
    for di in range(config.MAX_Len):
        decoder_output, decoder_hidden = decoder_attention(decoder_input, decoder_hidden, encoder_output)
        topv, topi = decoder_output.data.topk(1)
        ni = topi[0][0]
        if ni == EOS_token:
            decoded_words.append('<EOS>')
            break
        else:
            decoded_words.append(output_lang.index2word[ni])
        # 把当前的输出当做输入
        decoder_input = Variable(torch.LongTensor([ni]))
        decoder_input = decoder_input.cuda() if config.use_gpu else decoder_input
        
    # 对 decoded_words 进行连接，输出结果
    output_sentence = ' '.join(decoded_words)
    print('<', output_sentence)
    print('')

In [ ]:
for i in range(10):
    sampling(encoder, decoder)